In [1]:
import numpy as np
import pandas as pd

data_mahasiswa = pd.read_csv("dataMahasiswa.csv")
data_mahasiswa

,mhs,ipk,gaji
0,A,3.00,10
1,B,2.99,1


In [4]:
# data ipk
data_ipk = np.array(data_mahasiswa)
data_ipk = data_ipk[:,[0,1]]

ipk_A = np.sum(data_ipk[0:1,1], axis=0)
ipk_B = np.sum(data_ipk[1:2,1], axis=0)

print("ipk_A: ", ipk_A)
print("ipk_B: ", ipk_B)

ipk_A:  3.0
ipk_B:  2.99


In [7]:
# data gaji
data_gaji = np.array(data_mahasiswa)
data_gaji = data_gaji[:,[0,2]]

gaji_A = np.sum(data_gaji[0:1,1], axis=0)
gaji_B = np.sum(data_gaji[1:2,1], axis=0)

print("gaji_A: ", gaji_A)
print("gaji_B: ", gaji_B)

gaji_A:  10
gaji_B:  1


In [8]:
%pip install -U scikit-fuzzy

  Using cached scikit_fuzzy-0.4.2-py3-none-any.whl
  Using cached scipy-1.13.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
Using cached scipy-1.13.0-cp312-cp312-win_amd64.whl (45.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\Users\\asus\\OneDrive\\Documents\\UNILA\\Semester 4\\Kecerdasan Buatan\\FUZZY-LOGIC\\myvenv\\Lib\\site-packages\\scipy-1.13.0-cp312-cp312-win_amd64.whl'

